Import

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import sklearn
from sklearn import svm, preprocessing 
import warnings

Reading Data

In [6]:
data = pd.read_csv("../../data_sets/Volumetric_features.csv")

df = pd.DataFrame(data, columns = data.columns[:-1])
df = df.drop(['S.No','Age'], axis=1)

df.head()
df.describe()


scaler = StandardScaler()
x = scaler.fit_transform(df)
n = 20
pca = PCA(n_components=n)
pca_dataF = pca.fit_transform(x)

labels = df.columns.values.tolist()
label_index = [np.abs(pca.components_[i]).argmax() for i in range(n)]
columns = [labels[label_index[i]] for i in range(n)]

pca_df = pd.DataFrame(data=pca_dataF, columns=columns)
print(pca_df.head)
pca_df.head()

<bound method NDFrame.head of       rh_MeanThickness_thickness  CerebralWhiteMatterVol  \
0                       1.754401                1.293658   
1                       1.417516                1.506790   
2                       2.060537                1.356490   
3                       4.321472                1.316558   
4                       3.432616                1.645475   
...                          ...                     ...   
4221                    3.508241                2.349694   
4222                    4.445945               -2.409494   
4223                    8.016491                2.326573   
4224                   -0.596625                2.547032   
4225                   -2.307309               -3.481649   

      Left-Lateral-Ventricle  lh_pericalcarine_thickness  SurfaceHoles  \
0                   1.400041                   -1.478853     -1.847097   
1                   1.653998                   -0.614319     -1.384144   
2                   1.48902

,rh_MeanThickness_thickness,CerebralWhiteMatterVol,Left-Lateral-Ventricle,lh_pericalcarine_thickness,SurfaceHoles,CC_Posterior,rh_caudalanteriorcingulate_thickness,CC_Posterior,Right-Caudate,lh_parahippocampal_thickness,MaskVol-to-eTIV,Brain-Stem,Left-vessel,Right-vessel,non-WM-hypointensities,rh_isthmuscingulate_thickness,5th-Ventricle,5th-Ventricle,5th-Ventricle,5th-Ventricle
0,1.754401,1.293658,1.400041,-1.478853,-1.847097,2.545767,-1.131810,-0.404930,-0.381592,1.589589,-0.191708,0.146054,-1.584993,0.605123,-0.364399,0.538132,-0.195776,-0.252385,0.786202,0.282347
1,1.417516,1.506790,1.653998,-0.614319,-1.384144,2.369572,-1.434910,-0.853649,-0.780437,1.752106,0.198117,0.362418,-1.472042,0.258926,-0.813482,0.255137,-0.610386,-0.147156,1.302750,0.289107
2,2.060537,1.356490,1.489023,-1.055988,-1.174589,2.165927,-2.161509,-0.812870,-1.134761,1.503610,0.084286,0.335938,-0.927496,0.727237,-0.824967,0.487809,-0.755372,0.280110,1.088763,-0.264713
3,4.321472,1.316558,1.100152,-0.240230,-1.374228,2.414806,-1.140274,-1.568228,-0.852910,1.348890,0.291676,-0.368548,-1.334032,0.553097,-0.284022,0.147491,-0.492888,0.086103,1.080340,-0.398838
4,3.432616,1.645475,2.126585,-0.804571,-1.321727,3.078528,-1.202790,-0.821232,-0.544174,1.568995,-0.067073,0.368240,-1.465207,1.257562,-0.811469,0.094560,-0.087863,-0.315034,1.441428,0.389351


Training Data

In [7]:
# Split for validation --> train, val, test = 80/15/5

#x_train, x_validation, y_train, y_validation = train_test_split(data.drop(['S.No','Age'], axis=1), data['Age'], test_size=0.20, random_state=33)

x_train, x_validation, y_train, y_validation = train_test_split(pca_df, data['Age'], test_size=0.20, random_state=33)

# (20% of total dataset -> 75% validation = 15% total, 25% validation = 5% total

x_val, x_test, y_val, y_test = train_test_split(x_validation, y_validation, test_size=0.25, random_state=33)

print("x_train shape is:",x_train.shape)
print("y_train shape is:",y_train.shape, "\n")
print("x_val shape is:",x_val.shape)
print("y_val shape is:",y_val.shape, "\n")
print("x_test shape is:",x_test.shape)
print("y_test shape is:",y_test.shape)

x_train shape is: (3380, 20)
y_train shape is: (3380,) 

x_val shape is: (634, 20)
y_val shape is: (634,) 

x_test shape is: (212, 20)
y_test shape is: (212,)


Ridge Regression 

In [8]:
rig_reg = linear_model.Ridge()
rig_reg.fit(x_train,y_train)
y_pred = rig_reg.predict(x_test)
print("accuracy: "+ str(rig_reg.score(x_test,y_test)*100) + "%")
print("Mean absolute error: {}".format(mean_absolute_error(y_test,y_pred)))
print("Mean squared error: {}".format(mean_squared_error(y_test,y_pred)))
print("Root mean squared error: {}".format(math.sqrt(mean_squared_error(y_test,y_pred))))
R2 = r2_score(y_test,y_pred)
print('R Squared: {}'.format(R2))
n=x_test.shape[0]
p=x_test.shape[1] - 1

adj_rsquared = 1 - (1 - R2) * ((n - 1)/(n-p-1))
print('Adjusted R Squared: {}'.format(adj_rsquared))

accuracy: 78.85291712780143%
Mean absolute error: 7.583056869105879
Mean squared error: 82.91532922500589
Root mean squared error: 9.10578548094594
R Squared: 0.7885291712780144
Adjusted R Squared: 0.7676023705190679
